In [1]:
import tensorflow as tf
from tensorflow import keras
import numpy as np

In [ ]:
def generate_time_series(batch_size, n_steps):
  freq1, freq2, offset1, offset2 = np.random.rand(4, batch_size, 1)
  time = np.linspace(0, 1, n_steps)
  series =  0.5 * np.sin((time - offset1) * (freq1 * 10 + 10))
  series +=  0.2 * np.sin((time - offset2) * (freq2 * 10 + 10))
  series +=  0.1 * np.random.rand(batch_size, n_steps) * 0.5
  return series[..., np.newaxis].astype(np.float32)

print(generate_time_series(32, 5).shape)

In [ ]:
n_steps = 50
series = generate_time_series(10000, n_steps + 1)
X_train, y_train = series[:7000, :n_steps], series[:7000, -1]
X_val, y_val = series[7000:9000, :n_steps], series[7000:9000, -1]
X_test, y_test = series[9000:, :n_steps], series[9000:, -1]

In [ ]:
# naive forecasting (last element in data as prediction)
# difficult to outperform

print(np.mean(keras.losses.mean_squared_error(y_val, X_val[:, -1])))

In [ ]:
# FFNN

model = keras.models.Sequential([
  keras.layers.Flatten(input_shape=[50, 1]),
  keras.layers.Dense(1),
])

model.compile(
  loss=keras.losses.MSE,
  metrics=[keras.metrics.MSE],
  optimizer=keras.optimizers.Adam()
)

model.fit(X_train, y_train, epochs=20)

print(tf.reduce_mean(keras.losses.MSE(y_test, model.predict(X_test))))

In [ ]:
# Simple RNN

model = keras.models.Sequential([
  keras.layers.SimpleRNN(1, input_shape=[None, 1])
])

model.compile(
  loss=keras.losses.MSE,
  metrics=[keras.metrics.MSE],
  optimizer=keras.optimizers.Adam()
)

model.fit(X_train, y_train, epochs=20)

print(tf.reduce_mean(keras.losses.MSE(y_test, model.predict(X_test))))

In [ ]:
# Deep RNN

model = keras.models.Sequential([
  keras.layers.SimpleRNN(20, input_shape=[None, 1], return_sequences=True),
  keras.layers.SimpleRNN(20, return_sequences=False),
  keras.layers.Dense(1),
])

model.compile(
  loss=keras.losses.MSE,
  metrics=[keras.metrics.MSE],
  optimizer=keras.optimizers.Adam()
)

model.fit(X_train, y_train, epochs=20)

print(tf.reduce_mean(keras.losses.MSE(y_test, model.predict(X_test))))

In [ ]:
# Using same model to predict multiple steps
new_steps = 10
series = generate_time_series(1, n_steps + new_steps)
X_new, Y_new = series[:, :n_steps], series[:, n_steps:n_steps + new_steps]
X = X_new
for step in range(new_steps):
  y_pred_one = model.predict(X[:, step:])[:, np.newaxis, :]
  X = np.concatenate((X, y_pred_one), axis=1)
y_pred = X[:, n_steps:]
print(y_pred)
print(tf.reduce_mean(keras.losses.MSE(Y_new, y_pred)))

In [ ]:
# Guessing 10 at a times
n_steps = 50
new_steps = 10
series = generate_time_series(10000, n_steps + new_steps)
X_train, y_train = series[:7000, :n_steps], series[:7000, -new_steps:]
X_val, y_val = series[7000:9000, :n_steps], series[7000:9000, -new_steps:]
X_test, y_test = series[9000:, :n_steps], series[9000:, -new_steps:]

print(series.shape)
print(X_train.shape, y_train.shape)
print(X_val.shape, y_val.shape)
print(X_test.shape, y_test.shape)

In [ ]:
model = keras.models.Sequential([
  keras.layers.SimpleRNN(20, input_shape=[None, 1], return_sequences=True),
  keras.layers.SimpleRNN(20, return_sequences=True),
  keras.layers.SimpleRNN(20, return_sequences=False),
  keras.layers.Dense(10),
])

model.compile(
  loss=keras.losses.MSE,
  metrics=[keras.metrics.MSE],
  optimizer=keras.optimizers.Adam()
)

model.fit(X_train, y_train, epochs=20)

In [ ]:
print(tf.reduce_mean(keras.losses.MSE(y_test.reshape(len(y_test), len(y_test[0])), model.predict(X_test))))

In [ ]:
# Sequence to sequence data
example_count = 10000
prediction_length = 10
Y = np.empty((example_count, n_steps, prediction_length))
for step_ahead in range(1, prediction_length + 1):
  Y[:, :, step_ahead - 1] = series[:, step_ahead:step_ahead + n_steps, 0]
y_train = Y[:7000]
y_val = Y[7000:9000]
y_test = Y[9000:]
print(y_train.shape, y_val.shape, y_test.shape)

In [ ]:
# Custom metric for evaluating since only the final output matters.
# During training it helped to use all outputs.
def last_time_step_mse(Y_true, Y_pred):
  return keras.metrics.mean_squared_error(Y_true[:, -1], Y_pred[:, -1])

# sequence to sequence model
model = keras.models.Sequential([
  keras.layers.SimpleRNN(20, return_sequences=True, input_shape=[None, 1]),
  keras.layers.SimpleRNN(20, return_sequences=True),
  keras.layers.TimeDistributed(keras.layers.Dense(10))
])

model.compile(
  loss=keras.losses.MSE,
  metrics=[last_time_step_mse],
  optimizer=keras.optimizers.Adam()
)

model.fit(X_train, y_train, epochs=20)
print(tf.reduce_mean(last_time_step_mse(y_test, model.predict(X_test))))

In [ ]:
# # NOT SURE WHY THIS DOESN'T WORK

# class LNSimpleRNNCell(keras.layers.Layer):
#   def __init__(self, units, activation='tanh', **kwargs):
#     super().__init__(**kwargs)
#     self.state_size = units
#     self.output_size = units
    
#     self.layer_norm = keras.layers.LayerNormalization()
#     self.activation = keras.activations.get(activation)
  
#   def build(self, batch_input_shape):
#     print(batch_input_shape)
#     self.simpleRNNCell = keras.layers.SimpleRNN(self.output_size, activation=None, input_shape=batch_input_shape)
#     super().build(batch_input_shape)
  
#   def call(self, inputs, states):
#     new_outputs, new_states = self.simpleRNNCell(inputs, states)
#     normalized_outputs = self.activation(self.layer_norm(new_outputs))
#     return normalized_outputs, [normalized_outputs]
    

In [ ]:
# # WON'T WORK BECAUSE OF ABOVE CELL
# model = keras.models.Sequential([
#   keras.layers.RNN(LNSimpleRNNCell(20), return_sequences=True, input_shape=[None, 1]),
#   keras.layers.RNN(LNSimpleRNNCell(20), return_sequences=True),
#   keras.layers.TimeDistributed(keras.layers.Dense(10))
# ])

# model.compile(
#   loss=keras.losses.MSE,
#   metrics=[last_time_step_mse],
#   optimizer=keras.optimizers.Adam()
# )

# model.fit(X_train, y_train, epochs=20)
# print(tf.reduce_mean(last_time_step_mse(y_test, model.predict(X_test))))

In [ ]:
model = keras.models.Sequential([
  keras.layers.LSTM(20, return_sequences=True),
  keras.layers.LSTM(20, return_sequences=True),
  keras.layers.TimeDistributed(keras.layers.Dense(10))
])

model.compile(
  loss=keras.losses.MSE,
  metrics=[last_time_step_mse],
  optimizer=keras.optimizers.Adam()
)

model.fit(X_train, y_train, epochs=20)
print(tf.reduce_mean(last_time_step_mse(y_test, model.predict(X_test))))

In [ ]:
model = keras.models.Sequential([
  keras.layers.Conv1D(filters=20, kernel_size=4, strides=2, padding='valid'),
  keras.layers.GRU(20, return_sequences=True),
  keras.layers.GRU(20, return_sequences=True),
  keras.layers.TimeDistributed(keras.layers.Dense(10))
])

model.compile(
  loss=keras.losses.MSE,
  metrics=[last_time_step_mse],
  optimizer=keras.optimizers.Adam()
)

model.fit(X_train, y_train[:, 3::2], epochs=20)
print(tf.reduce_mean(last_time_step_mse(y_test[:, 3::2], model.predict(X_test))))

In [ ]:
model = keras.models.Sequential()
model.add(keras.layers.InputLayer(input_shape=[None, 1]))
for dilation in [1, 2, 4, 8] * 2:
  model.add(keras.layers.Conv1D(filters=20, kernel_size=2, padding='causal', activation='relu', dilation_rate=dilation))
model.add(keras.layers.Conv1D(filters=10, kernel_size=1))

model.compile(
  loss=keras.losses.MSE,
  metrics=[last_time_step_mse],
  optimizer=keras.optimizers.Adam()
)

model.fit(X_train, y_train, epochs=20)
print(tf.reduce_mean(last_time_step_mse(y_test, model.predict(X_test))))

In [ ]:
print(y_test.shape, model.predict(X_test).shape)

In [ ]:
import tensorflow_datasets as tfds

dataset = tfds.load('imagenet_sketch', as_supervised=True)
dataset = dataset['test']

In [ ]:
train = dataset.take(40000)
val = dataset.skip(40000).take(5000)
test = dataset.skip(45000)
print(len(train), len(val), len(test))